In [86]:
import numpy as np
import pickle
import os



from utilities.dataset import distribution_data_sampler
from utilities import constants


from utilities.custom_layers import BaseEncoder, BaseDecoder

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

n_samples = 10000

2023-04-15 15:44:20.735287: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.10.1


# Loss function

In [2]:
from itertools import product

In [64]:
def exp_func(array, e_b):
    summ = 0
    for i, b in enumerate(array):
        summ += 2**(len(array)-1-i)*int(b)
    return  summ - e_b

def mant_func(array, e):
    summ = int(any((int(x) for x in e)))
    for i, b in enumerate(array, start=1):
        summ += 2**(-i)*int(b)
    return summ

def float_from_str(s, e ,m, e_b):
    sign_val = (-1)**int(s)
    exp_val =   exp_func(e, e_b)
    mant_val = mant_func(m, e)
    float_val = sign_val * 2**exp_val * mant_val
    return float_val, sign_val, exp_val, mant_val

In [121]:
x = (np.arange(15)-7)/2
x

array([-3.5, -3. , -2.5, -2. , -1.5, -1. , -0.5,  0. ,  0.5,  1. ,  1.5,
        2. ,  2.5,  3. ,  3.5])

In [132]:
np.ldexp(np.array([0.2]), np.array([1]))

array([0.4])

In [123]:
np.frexp(x)

(array([-0.875, -0.75 , -0.625, -0.5  , -0.75 , -0.5  , -0.5  ,  0.   ,
         0.5  ,  0.5  ,  0.75 ,  0.5  ,  0.625,  0.75 ,  0.875]),
 array([2, 2, 2, 2, 1, 1, 0, 0, 0, 1, 1, 2, 2, 2, 2], dtype=int32))

In [111]:
[2**x*(0.999) for x in range(4)][::-1]

[7.992, 3.996, 1.998, 0.999]

In [106]:
[x-7 for x in (range(0, 16))]

[-7, -6, -5, -4, -3, -2, -1, 0, 1, 2, 3, 4, 5, 6, 7, 8]

In [91]:
2**np.geomspace(0.001, 1, 16)

array([1.00069339, 1.00109917, 1.00174262, 1.00276328, 1.00438304,
       1.00695555, 1.01104621, 1.01756353, 1.02797895, 1.04470509,
       1.07177346, 1.11611781, 1.19018731, 1.31777784, 1.54859227,
       2.        ])

In [84]:
bool_array = [0, 1]
exp = 2
mant = 1
sign = 1
exp_bias = 2**(exp-1)-1

tuple_to_str = lambda x: ''.join(map(str, (x)))

for tup in product('01', repeat=exp + mant + sign):
    s = tup[0]
    e = tup[1:exp+1]
    m = tup[-mant:]
    float_val, sign_val, exp_val, mant_val = float_from_str(s, e, m, exp_bias)
    
    print(f'{s} {tuple_to_str(e)} {tuple_to_str(m):<5} {float_val}')
#     print(f'{sign_val} {exp_val:<4} {mant_val}')

0 00 0     0.0
0 00 1     0.25
0 01 0     1.0
0 01 1     1.5
0 10 0     2.0
0 10 1     3.0
0 11 0     4.0
0 11 1     6.0
1 00 0     -0.0
1 00 1     -0.25
1 01 0     -1.0
1 01 1     -1.5
1 10 0     -2.0
1 10 1     -3.0
1 11 0     -4.0
1 11 1     -6.0


In [ ]:
bool_array = [0, 1]
for s in bool_array:
    for e2 in bool_array:
        for e1 in bool_array:
            for m in bool_array:
                val = (-1)**s * (2**(2*e2 + 1*e1 )+ 1) * (1 + m*1/2)


In [ ]:
y = np.array([8, 8, 8, 8, 8, 8, 8, 8, 
              32, 32, 32, 32, 32, 32, 32, 32,
              512, 512, 512, 512, 512, 512, 512, 512]).reshape(-1, 1)

y_hat = np.array([8, 16, -8, -16, 8.125, 16.25, -8.125, -16.25,
                  32, 64, -32, -64, 32.5, 65, -32.5, -65, 
                  512, 1024, -512, -1024, 520, 1040, -520, -1040]).reshape(-1, 1)


# y = np.array([1, 2, 4, 8, 16]).reshape(-1, 1)

# y_hat = np.array([0.5, 1, 2, 4, 8]).reshape(-1, 1)

be_layer = BaseEncoder()

y_be = be_layer(y).numpy()
y_hat_be = be_layer(y_hat).numpy()
y_hat_be_raw = y_hat_be.copy()
np.set_printoptions(suppress=True)
np.place(y_hat_be, y_hat_be == 1, 0.999999)
np.place(y_hat_be, y_hat_be == 0, 0.000001)

In [ ]:
def bce(y_be, y_hat):
    return np.multiply(y_be, np.log(y_hat)) + np.multiply(1 - y_be, np.log(1 - y_hat))

In [ ]:
def bece_func(y_be, y_hat, y_sign, y_hat_sign):
    y_sign = np.multiply(np.ones(y_be.shape), y_sign)
    y_hat_sign = np.multiply(np.ones(y_hat.shape), y_hat_sign)

    neg_one = np.multiply(np.multiply(y_be, y_sign), np.log(np.multiply(y_hat, y_hat_sign)))
    pos_one = np.multiply(np.multiply(y_be, (1 - y_sign)), np.log(np.multiply(y_hat,(1 - y_hat_sign))))
##################### do not uncomment - it makes the loss equally disregard of its value 
##################### loss(512, -512)= loss(8, -8)
#     neg_zero = np.multiply(np.multiply(1 - y_be, y_sign), np.log(np.multiply(1 - y_hat, y_hat_sign)))
#     pos_zero = np.multiply(np.multiply(1 - y_be, (1 - y_sign)), np.log(np.multiply(1 - y_hat,(1 - y_hat_sign))))
#     return -(neg_zero + pos_zero + neg_one + pos_one)
#####################
    zeros = np.multiply(1 - y_be, np.log(1 - y_hat))
    return -(zeros + neg_one + pos_one)


In [ ]:
def preprocess(y, y_hat):
    be_layer = BaseEncoder()

    y_be = be_layer(y).numpy()
    y_hat_be = be_layer(y_hat).numpy()

    np.place(y_hat_be, y_hat_be == 1, 0.999999)
    np.place(y_hat_be, y_hat_be == 0, 0.000001)
    
    return y_be, y_hat_be

In [ ]:
def new_loss_func(y, y_hat):
    y_mant = y[:, :, 9:]
    y_exp = y[:, :, 1:9]
    y_sign = y[:, :, :1]


    y_hat_mant = y_hat[:, :, 9:]
    y_hat_exp = y_hat[:, :, 1:9]
    y_hat_sign = y_hat[:, :, :1]
    
    mantissa_coefs = (np.ones((y_mant.shape[0], y_mant.shape[1], 1))*2)**((np.arange(-11, 12)[::-1]).reshape(1, -1))
    mantissa_bce = bce(y_mant, y_hat_mant)
    mantissa_part = (1 + np.multiply(mantissa_coefs, mantissa_bece)).sum(axis=-1)


    exponent_coefs = (np.ones((y_exp.shape[0], y_exp.shape[1], 1))*2)**((np.arange(1, 9))[::-1].reshape(1, -1))
    exponent_bece = bece_func(y_exp, y_hat_exp, y_sign, y_hat_sign)
    exponent_part = np.multiply(exponent_coefs, exponent_bece).sum(axis=-1) 
    

In [ ]:
y_mant = y_be[:, :, 9:]
y_hat_mant = y_hat_be[:, :, 9:]

y_sign = y_be[:, :, :1]
y_hat_sign = y_hat_be[:, :, :1]

In [ ]:
mantissa_bece = bce(y_mant, y_hat_mant)
mantissa_coefs = (np.ones((y_mant.shape[0], y_mant.shape[1], 1))*2)**-((np.arange(1, 24)[::-1]).reshape(1, -1))
mantissa_part = (1 - np.multiply(mantissa_bece, mantissa_coefs).sum(axis=-1))

In [ ]:
y_exp = y_be[:, :, 1:9]
y_hat_exp = y_hat_be[:, :, 1:9]

In [ ]:
exponent_coefs = (np.ones((y_exp.shape[0], y_exp.shape[1], 1))*2)**((np.arange(1, 9))[::-1].reshape(1, -1))
exponent_bce = bece_func(y_exp, y_hat_exp, y_sign, y_hat_sign)
exponent_part = np.multiply(exponent_coefs, exponent_bce).sum(axis=-1) 

In [ ]:
new_loss = np.log2(exponent_part) + np.log2(mantissa_part)

In [ ]:
def loss_func(y, y_hat):
    
    y_mant = y[:, :, 9:]
    y_exp = y[:, :, 1:9]
    y_sign = y[:, :, :1]


    y_hat_mant = y_hat[:, :, 9:]
    y_hat_exp = y_hat[:, :, 1:9]
    y_hat_sign = y_hat[:, :, :1]
    
    
#     mantissa_coefs = (np.ones((y_mant.shape[0], y_mant.shape[1], 1))*2)**-((np.arange(0, 23)).reshape(1, -1))
    mantissa_coefs = (np.ones((y_mant.shape[0], y_mant.shape[1], 1))*2)**((np.arange(-11, 12)[::-1]).reshape(1, -1))
    mantissa_bece = bece_func(y_mant, y_hat_mant, y_sign, y_hat_sign)
    mantissa_part = np.log2(np.multiply(mantissa_coefs, mantissa_bece).sum(axis=-1))
#     mantissa_part = (1 + np.multiply(mantissa_coefs, mantissa_bece)).sum(axis=-1)


    exponent_coefs = (np.ones((y_exp.shape[0], y_exp.shape[1], 1))*2)**((np.arange(1, 9))[::-1].reshape(1, -1))
    exponent_bece = bece_func(y_exp, y_hat_exp, y_sign, y_hat_sign)
    exponent_part = np.multiply(exponent_coefs, exponent_bece).sum(axis=-1) 
    
    return exponent_part, mantissa_part 

In [ ]:
# exponent_part, mantissa_part = loss_func(y_be, y_hat_be)
# final_loss = exponent_part + mantissa_part
df = pd.DataFrame(np.concatenate((y, y_hat, (np.abs(y_be - y_hat_be_raw)).sum(axis=-1),((y - y_hat)**2).astype(np.float32), exponent_part, mantissa_part, new_loss), axis=-1), 
             columns=['y', 'y_hat', 'binary diff number', 'mse', 'exponent part', 'mantissa part', 'new loss'])

In [ ]:
df.sort_values("new loss")